In [6]:
#import datetime
import numpy as np
#import matplotlib.pyplot as plt
#import pandas as pd
#from mpl_toolkits.axes_grid1 import make_axes_locatable
from tensorflow import keras
from tensorflow.keras import layers
#from transform_output_format import get_2D_output, get_4D_output

#pd.set_option ('display.max_columns', None)

In [7]:
# Load data from raw_data
data = np.load('../raw_data/baseline.npz', allow_pickle=True)

In [8]:
y = pd.read_csv('../raw_data/y_train.csv')

NameError: name 'pd' is not defined

In [ ]:
y=get_4D_output(y_train)
print(y.shape)

In [9]:
GHI=data['GHI']
CLS=data['CLS']

In [11]:
X_train=np.zeros((4,4,51,51,1))
for j in range (4):
    for i in range (4):
        for k in range (51):
            for h in range (51):
                for z in range (1):
                        X_train[j,i,k,h,z]=GHI[j,i,k+15,h+15]               
X_train.shape

(4, 4, 51, 51, 1)

In [ ]:
X_val=np.zeros((1,4,51,51,1))
for j in range (1):
    for i in range (4):
        for k in range (51):
            for h in range (51):
                for z in range (1):
                        X_train[j,i,k,h,z]=GHI[j,i,k+15,h+15]               
X_val.shape

In [ ]:
y_train=np.zeros((4,4,51,51,1))
for j in range (4):
    for i in range (4):
        for z in range (51):
            for k in range (51):
                    for h in range (1):
                        y_train[j,i,k,h,z]=y[j,i,k,h]
y_train.shape

In [ ]:
y_val=np.zeros((1,4,51,51,1))
for j in range (1):
    for i in range (4):
        for z in range (51):
            for k in range (51):
                    for h in range (1):
                        y_train[j,i,k,h,z]=y[j,i,k,h]
y_val.shape

In [ ]:
inp=layers.Input(shape=(None, *X_model.shape[2:]))

x= layers.ConvLSTM2D(filters=64, kernel_size=(5,5), 
                     padding='same',
                     return_sequences=True, 
                     activation='relu',
                    )(inp)
#x=layers.BatchNormalization()(x)
x=layers.Conv3D(filters=1, kernel_size=(3,3,3),
                activation="sigmoid", 
                padding="same")(x)
model=keras.models.Model(inp,x)
model.compile(loss=keras.losses.binary_crossentropy, 
              optimizer=keras.optimizers.Adam(),
              )

In [ ]:
#Define some callbacks to improve training
early_stopping=keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
reduce_lr=keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=5)

#Define modificable training hyperparameters
epochs=20
batch_size=5

#Fit the model to the training data
model.fit(X_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(X_val, y_val)
    callbacks=[early_stopping, reduce_lr],
)

